SET100 Data: Feature Set 01

Use class StockFS as container for each stock.  

Start with list of tickers

Join with Sector data  
Ticker	Sector	Industry  


Eikon data  
Price	MV	VA	VO	DY  

Merge with YahooFinance  
High	Low	Open	Close	Volume	Adj Close



208164 rows × 26 columns   _fs1.csv
12M	_fs1.csv.gz

In [1]:
import sys, os, os.path
if not 'EXPERIMENT_HOME' in os.environ: # Configure local paths if not already defined
    ML_HOME = os.path.abspath(os.path.join("/home/jovyan", "ml_home")) # ML workspace
    EXPERIMENT_HOME = os.path.abspath(os.path.join(ML_HOME, "..")) # Experiment workspace
    DATA_HOME = os.path.abspath(os.path.join(EXPERIMENT_HOME,"dataset")) # Dataset location
    os.environ["EXPERIMENT_HOME"] = EXPERIMENT_HOME
    os.environ["DATA_HOME"] = DATA_HOME
    sys.path.insert(0, EXPERIMENT_HOME)
    sys.path.insert(0, ML_HOME) # Add to path so can load our library
    os.chdir(EXPERIMENT_HOME) # Change working directory to experiment workspace
print("Experiment Home: ", os.path.abspath(os.curdir), "; Data Home:", DATA_HOME)

Experiment Home:  /home/jovyan ; Data Home: /home/jovyan/dataset


In [2]:
from datetime import datetime
import pandas as pd
import pandas_datareader as pdr
import datetime as dt
import xlrd
import matplotlib.pyplot as plt
from preprocessing import add_feature_set_1
%matplotlib inline


In [3]:
os.chdir("/home/jovyan/dataset/set100.data")
stocks = pd.read_csv('_directory.csv', index_col=0)
print("Stocks count:", len(stocks))
stocks.tail()

Stocks count: 150


,company_name,industry,sector,symbol
ticker,,,,
VIBHA,VIBHAVADI MEDICAL CENTER PUBLIC COMPANY LIMITED,Services,Health Care Services,VIBHA
VNG,VANACHAI GROUP PUBLIC COMPANY LIMITED,Property & Construction,Construction Materials,VNG
WHA,WHA CORPORATION PUBLIC COMPANY LIMITED,Property & Construction,Property Development,WHA
WHAUP,WHA UTILITIES AND POWER PUBLIC COMPANY LIMITED,Resources,Energy & Utilities,WHAUP
WORK,WORKPOINT ENTERTAINMENT PUBLIC COMPANY LIMITED,Services,Media & Publishing,WORK


In [4]:
period_start='2015-01-01'
period_end='2019-12-31'
series = {}
for s in stocks.symbol.values:
    df = pd.read_csv("%s.csv" % s, index_col=0)
    # Add column ticker
    df.loc[:,('Date')] = df.index
    df['Ticker'] = s
    
    # Add sector column
    sector = stocks[stocks['symbol']==s].sector.values[0]
    df['Sector'] = sector

    # Add industry column
    industry = stocks[stocks['symbol']==s].industry.values[0]
    df['Industry'] = industry
    
    # Add to dictionary
    series[s] = df[period_start:period_end]

In [5]:
# Select single stock series for analysis    
ticker = "SCB"
df = series[ticker]
df.index = pd.to_datetime(df.index)
df

,Price,MV,VA,VO,DY,Date,Ticker,Sector,Industry
Date,,,,,,,,,
2015-01-01,182.0,617764.2,NaN,NaN,2.88,2015-01-01,SCB,Banking,Financials
2015-01-02,182.0,617764.2,NaN,NaN,2.88,2015-01-02,SCB,Banking,Financials
2015-01-05,179.0,607581.3,1035117.0,5752.3,2.93,2015-01-05,SCB,Banking,Financials
2015-01-06,175.0,594004.1,1822522.0,10431.0,3.00,2015-01-06,SCB,Banking,Financials
2015-01-07,178.0,604186.9,792808.3,4456.9,2.95,2015-01-07,SCB,Banking,Financials
...,...,...,...,...,...,...,...,...,...
2019-12-25,119.5,405771.6,457841.1,3841.8,4.60,2019-12-25,SCB,Banking,Financials
2019-12-26,119.0,404073.9,630049.9,5295.4,4.62,2019-12-26,SCB,Banking,Financials
2019-12-27,119.5,405771.6,667488.9,5586.6,4.60,2019-12-27,SCB,Banking,Financials


In [6]:
# Stack dataframes together
# https://stackoverflow.com/questions/29351840/stack-two-pandas-data-frames
rows = []
for k, d in series.items():
    rows.append(d)
df = pd.concat(rows, ignore_index=True)
df

,Price,MV,VA,VO,DY,Date,Ticker,Sector,Industry
0,4.30,20855.00,NaN,NaN,0.00,2015-01-01,AAV,Transportation & Logistics,Services
1,4.30,20855.00,NaN,NaN,0.00,2015-01-02,AAV,Transportation & Logistics,Services
2,4.30,20855.00,71287.9,16567.4,0.00,2015-01-05,AAV,Transportation & Logistics,Services
3,4.54,22018.99,265539.0,59230.2,0.00,2015-01-06,AAV,Transportation & Logistics,Services
4,4.58,22212.99,110609.1,24048.5,0.00,2015-01-07,AAV,Transportation & Logistics,Services
...,...,...,...,...,...,...,...,...,...
193711,13.60,6005.22,25806.5,1918.2,3.31,2019-12-25,WORK,Media & Publishing,Services
193712,13.90,6137.69,26514.5,1907.0,3.24,2019-12-26,WORK,Media & Publishing,Services
193713,14.10,6226.00,22389.9,1587.2,3.19,2019-12-27,WORK,Media & Publishing,Services
193714,14.00,6181.84,2242.3,159.0,3.21,2019-12-30,WORK,Media & Publishing,Services


In [7]:
# Pivot table: Value=Price
ptable = df.pivot_table(values='Price', index='Date', columns=['Industry', 'Sector', 'Ticker'])
ptable

Industry   Agro & Food Industry                                              \
Sector             Agribusiness        Food & Beverage                        
Ticker                     GFPT    STA             CBG    CPF   ICHI   KTIS   
Date                                                                          
2015-01-01                 17.8  12.33           33.50  27.25  20.00  11.60   
2015-01-02                 17.8  12.33           33.50  27.25  20.00  11.60   
2015-01-05                 17.5  12.43           33.00  27.25  20.00  12.00   
2015-01-06                 17.4  12.43           31.25  26.50  19.90  11.90   
2015-01-07                 17.5  12.53           30.25  26.75  20.30  11.90   
...                         ...    ...             ...    ...    ...    ...   
2019-12-25                 11.9   9.80           82.75  27.75   4.82   4.16   
2019-12-26                 12.1   9.95           83.75  27.75   5.00   4.18   
2019-12-27                 12.4  10.10           84.50  27.50   4.92   4.16   
2019-12-30                 12.3  10.00           84.00  27.50   5.05   4.20   
2019-12-31                 12.3  10.00           84.00  27.50   5.05   4.20   

Industry                                ...            Technology  \
Sector                                  ... Electronic Components   
Ticker          M  MALEE   MINT    OSP  ...                   SVI   
Date                                    ...                         
2015-01-01  58.50  13.75  29.10    NaN  ...                  4.00   
2015-01-02  58.50  13.75  29.10    NaN  ...                  4.00   
2015-01-05  58.25  13.75  28.43    NaN  ...                  4.00   
2015-01-06  57.50  13.75  27.31    NaN  ...                  4.10   
2015-01-07  58.00  13.87  27.76    NaN  ...                  4.16   
...           ...    ...    ...    ...  ...                   ...   
2019-12-25  70.00   6.55  35.21  39.75  ...                  3.46   
2019-12-26  69.00   6.60  35.21  40.00  ...                  3.50   
2019-12-27  70.00   6.60  35.21  40.25  ...                  3.56   
2019-12-30  70.50   6.50  35.46  40.50  ...                  3.60   
2019-12-31  70.50   6.50  35.46  40.50  ...                  3.60   

Industry                                                                     \
Sector     Information & Communication Technology                             
Ticker                                     ADVANC   DTAC INTUCH   JAS JMART   
Date                                                                          
2015-01-01                                  251.0  96.50  78.75  6.62  5.41   
2015-01-02                                  251.0  96.50  78.75  6.62  5.41   
2015-01-05                                  249.0  95.50  78.00  6.62  5.46   
2015-01-06                                  247.0  94.50  77.50  6.58  5.41   
2015-01-07                                  253.0  93.25  79.50  6.67  5.35   
...                                           ...    ...    ...   ...   ...   
2019-12-25                                  215.0  53.25  57.50  5.00  8.05   
2019-12-26                                  215.0  53.75  57.50  4.98  8.30   
2019-12-27                                  214.0  53.25  57.25  4.98  8.25   
2019-12-30                                  213.0  53.25  57.25  5.00  8.35   
2019-12-31                                  213.0  53.25  57.25  5.00  8.35   

Industry                                
Sector                                  
Ticker     SAMART SAMTEL  THCOM   TRUE  
Date                                    
2015-01-01  39.25  22.70  34.75  10.90  
2015-01-02  39.25  22.70  34.75  10.90  
2015-01-05  38.00  23.60  35.75  11.29  
2015-01-06  38.00  23.80  35.50  11.39  
2015-01-07  39.00  25.25  35.25  11.39  
...           ...    ...    ...    ...  
2019-12-25   6.85   7.90   3.84   4.62  
2019-12-26   6.80   7.90   3.78   4.66  
2019-12-27   6.80   7.90   3.76   4.62  
2019-12-30   6.80   7.95   3.86   4.60  
2019-12-31   6.80 

# Merge with Yahoo Finance

In [8]:
class StockFS1:
    """Feature set 1"""
    def __init__(self, ticker):
        self.ticker = ticker
        df = series[ticker]
        df.index = pd.to_datetime(df.index)
        self.df = df
        
        self.outfile = "%s_FS1.csv" % self.ticker
        
        # YahooFinance
        self.merge_yahoofinance()
        
        # Add Feature Set 01
        # add_feature_set_1(self.df)
        
    def merge_yahoofinance(self):
        symbol = self.ticker
        if symbol == 'GOLD': # Exception: change to FPT
            symbol = 'FPT'
        srcfile = os.path.join(DATA_HOME,"historical","%s.csv" % symbol)
        df_yahoo = pd.read_csv(srcfile)
        df_yahoo = df_yahoo.set_index('Date')
        df_yahoo.index = pd.to_datetime(df_yahoo.index)
        
        df_res = self.df.merge(df_yahoo, left_on=self.df.index, right_on=df_yahoo.index)
        df_res = df_res.set_index('key_0')
        df_res.index.rename('Date', inplace=True)
        self.df = df_res
        return True

# Compute features and save

In [9]:
stocks = []
for t in series.keys():
    s1 = StockFS1(t)
    print("Merged %s." % t)
    stocks.append(s1.df)

Merged AAV.
Merged ADVANC.
Merged AEONTS.
Merged AMATA.
Merged ANAN.
Merged AOT.
Merged AP.
Merged ASP.
Merged AWC.
Merged BA.
Merged BANPU.
Merged BAY.
Merged BBL.
Merged BCH.
Merged BCP.
Merged BCPG.
Merged BDMS.
Merged BEAUTY.
Merged BEC.
Merged BEM.
Merged BGRIM.
Merged BH.
Merged BIG.
Merged BJC.
Merged BJCHI.
Merged BLA.
Merged BLAND.
Merged BPP.
Merged BTS.
Merged CBG.
Merged CENTEL.
Merged CHG.
Merged CK.
Merged CKP.
Merged COM7.
Merged CPALL.
Merged CPF.
Merged CPN.
Merged DELTA.
Merged DEMCO.
Merged DTAC.
Merged EA.
Merged EGCO.
Merged EPG.
Merged ERW.
Merged ESSO.
Merged GOLD.
Merged GFPT.
Merged GGC.
Merged GL.
Merged GLOBAL.
Merged GPSC.
Merged GULF.
Merged GUNKUL.
Merged HANA.
Merged HMPRO.
Merged ICHI.
Merged INTUCH.
Merged IRPC.
Merged ITD.
Merged IVL.
Merged JAS.
Merged JMART.
Merged JMT.
Merged JWD.
Merged KAMART.
Merged KBANK.
Merged KCE.
Merged KKP.
Merged KTB.
Merged KTC.
Merged KTIS.
Merged LH.
Merged LHFG.
Merged LOXLEY.
Merged LPN.
Merged M.
Merged MAJOR.
Merged

In [10]:
# Stack dataframes together
# https://stackoverflow.com/questions/29351840/stack-two-pandas-data-frames

df_all = pd.concat(stocks, ignore_index=True)
df_all

,Price,MV,VA,VO,DY,Date,Ticker,Sector,Industry,High,Low,Open,Close,Volume,Adj Close
0,4.30,20855.00,71287.9,16567.4,0.00,2015-01-05,AAV,Transportation & Logistics,Services,4.32,4.28,4.30,4.30,16567300.0,3.756939
1,4.54,22018.99,265539.0,59230.2,0.00,2015-01-06,AAV,Transportation & Logistics,Services,4.56,4.34,4.34,4.54,59230000.0,3.966629
2,4.58,22212.99,110609.1,24048.5,0.00,2015-01-07,AAV,Transportation & Logistics,Services,4.64,4.56,4.62,4.58,24048400.0,4.001577
3,4.62,22407.00,70601.2,15339.9,0.00,2015-01-08,AAV,Transportation & Logistics,Services,4.64,4.58,4.62,4.62,15339900.0,4.036525
4,4.72,22892.00,237218.0,50678.6,0.00,2015-01-09,AAV,Transportation & Logistics,Services,4.72,4.62,4.66,4.72,50678500.0,4.123896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178037,13.40,5916.91,43853.1,3283.1,3.36,2019-12-24,WORK,Media & Publishing,Services,13.70,13.00,13.60,13.40,3282900.0,12.420717
178038,13.60,6005.22,25806.5,1918.2,3.31,2019-12-25,WORK,Media & Publishing,Services,13.80,13.10,13.30,13.60,1918100.0,12.606101
178039,13.90,6137.69,26514.5,1907.0,3.24,2019-12-26,WORK,Media & Publishing,Services,14.10,13.60,13.60,13.90,1906900.0,12.884177
178040,14.10,6226.00,22389.9,1587.2,3.19,2019-12-27,WORK,Media & Publishing,Services,14.30,13.90,14.00,14.10,1587200.0,13.069561


In [11]:
df_all[df_all['Ticker']=='AAV']

,Price,MV,VA,VO,DY,Date,Ticker,Sector,Industry,High,Low,Open,Close,Volume,Adj Close
0,4.30,20855.00,71287.9,16567.4,0.00,2015-01-05,AAV,Transportation & Logistics,Services,4.32,4.28,4.30,4.30,16567300.0,3.756939
1,4.54,22018.99,265539.0,59230.2,0.00,2015-01-06,AAV,Transportation & Logistics,Services,4.56,4.34,4.34,4.54,59230000.0,3.966629
2,4.58,22212.99,110609.1,24048.5,0.00,2015-01-07,AAV,Transportation & Logistics,Services,4.64,4.56,4.62,4.58,24048400.0,4.001577
3,4.62,22407.00,70601.2,15339.9,0.00,2015-01-08,AAV,Transportation & Logistics,Services,4.64,4.58,4.62,4.62,15339900.0,4.036525
4,4.72,22892.00,237218.0,50678.6,0.00,2015-01-09,AAV,Transportation & Logistics,Services,4.72,4.62,4.66,4.72,50678500.0,4.123896
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1245,2.14,10379.00,6025.7,2797.2,15.65,2019-12-24,AAV,Transportation & Logistics,Services,2.18,2.14,2.16,2.14,2797100.0,2.140000
1246,2.16,10476.00,3756.8,1739.2,15.51,2019-12-25,AAV,Transportation & Logistics,Services,2.18,2.14,2.14,2.16,1739100.0,2.160000
1247,2.16,10476.00,5533.2,2558.2,15.51,2019-12-26,AAV,Transportation & Logistics,Services,2.18,2.14,2.18,2.16,2558100.0,2.160000
1248,2.20,10669.99,10248.2,4667.2,15.23,2019-12-27,AAV,Transportation & Logistics,Services,2.22,2.16,2.18,2.20,4667000.0,2.200000


In [12]:
# df_all.to_excel('Data.xlsx', sheet_name='Data')

In [13]:
!ls | grep Data

In [14]:
df_all.to_csv('data_v1.csv')

In [15]:
! pwd

/home/jovyan/dataset/set100.data
